In [1]:
from __future__ import print_function, division
import numpy as np
from random import Random, random
import time

class State(object):
    MUTATION_RATE = 0.08 # Play with this value, higher value means more chance of mutation
    def __init__(self, parents=None, generation = 1):
        r = Random()
        self._fitness = None
        self._probability = None
        self.generation = generation
        if parents==None:
            self.parents = None
            #self.state = [r.randint(1,8) for y in range(8)]
            # The following should also work
            #self.state = list(np.random.permutation(8) + 1)
            
            nrow, ncol = 1, 8
            uarray= (np.random.permutation(nrow * ncol) + 1)
            self.state=uarray.tolist()
            
        else:            
            parent1 = parents[0]
            parent2 = parents[1]
            self.parents = parents
            self.state = self.crossover(parent1, parent2)
            self.mutate()

    def fitness(self): 
        if not self._fitness:
            state = self.state
            
            horizontal_collisions = sum([state.count(col)-1 for col in state])/2
    
            diagonal_collisions = 0
            for i, col in enumerate(state):
                for j, diagonal in enumerate(state):
                    mod = abs(i-j)
                    if mod > 0: #we don't want to count the current queen in a collision with herself
                        if diagonal + mod == col or diagonal - mod == col:
                            diagonal_collisions += 1
            diagonal_collisions /= 2 #we were counting the diagonal collisions double
            self._fitness = int(28 - (horizontal_collisions + diagonal_collisions))
        return self._fitness

    def probability(self, population):
        if not self._probability:
            self._probability = self.fitness() / sum([x.fitness() for x in population])
        return self._probability

    def crossover_naive(self, parent1, parent2):
        r = Random()
        crossover_index = r.randint(0,8)
        left = parent1.state[0:crossover_index]
        right = parent2.state[crossover_index:8]
        left.extend(right)
        return left

    def crossover(self, parent1, parent2):
        r = Random()
        crossover_index = r.randint(0,8)
        left = parent1.state[0:crossover_index]
        
        right = parent2.state[crossover_index:8]
        right.extend(parent2.state[0:crossover_index])
        right.extend(parent1.state[crossover_index:8])
        
        j=len(left)
        k=0
        t=0
        while j<len(parent1.state):
            
            if k<len(right):
            
                if not (right[k] in left):
                        left.append(right[k])
                        j+=1
                k+=1        
            else : 
                a=len(parent1.state)-j
                pos=len(right)-(8-crossover_index)-(crossover_index)+1
                left.extend(right[pos:pos+a])
                j=len(parent1.state)
                
        return left
    
    def mutate_naive(self):
        r = Random()
        for i in range(len(self.state)):
            if random() < State.MUTATION_RATE:
                self.state[i] = r.randint(0,8)
    
    def mutate(self):
        k=0
        i=0
        pos=[]
        while (i < len(self.state)) and (k<=1):
            if random() < State.MUTATION_RATE:
                pos.append(i)
                k+=1
            i+=1
        if k==2:        
            ind1=pos[0]
            ind2=pos[1]
            temp=self.state[ind1]
            self.state[ind1]= self.state[ind2]
            self.state[ind2]=temp
            
    def __str__(self):
        # return repr(self.state)

        queen_symbol = '♛'
        blank_white_symbol = '▧'  # ▧, ▨, ▩, ▥, ▤, ▣
        blank_black_symbol = '☐'

        # _state = np.zeros((8,8), dtype = np.uint8)
        # Create a blank board with alternating 0 and 1
        _i = np.sum(np.indices((8,8)), axis = 0)
        _state = np.where(_i % 2 == 0, 0, 1)
      
        for i, j in enumerate(self.state):
            _state[j - 1, i] = 2

        _repr = "Generation: " + str(self.generation) + "\nFitness: " + str(self.fitness()) + "\n" + str(self.state) +  "\n"
        for h in range(8):
            for w in range(8):
                _repr += ([blank_white_symbol, blank_black_symbol, queen_symbol][_state[h, w]] + " ")
            _repr += "\n"
        return _repr

def pickRandomByProbability(populationByProbability):
    i = 0
    selectedState = None
    while selectedState == None:
        current = populationByProbability[i]
        if current[0] <= random():
            return current[1]
        if i+1 <= len(populationByProbability):
            i = 0
        else:
            i += 1

def generateNextPopulation(population, n):
    newPopulation = []
    while len(newPopulation) < n:
        populationByProbability = [(x.probability(population), x) for x in population]
        parent1 = pickRandomByProbability(populationByProbability)
        populationByProbability = [x for x in populationByProbability if x[1] != parent1]
        parent2 = pickRandomByProbability(populationByProbability)
        newPopulation.append(State((parent1, parent2), generation = parent1.generation + 1))
    return newPopulation



if __name__ == '__main__':
    populationSize = 50
    generation = 0
    population = [State(generation = generation) for x in range(populationSize)]
    
    while not 28 in [x.fitness() for x in population]:
        print("generation %d Max fitness: %d" % (generation, max([x.fitness() for x in population])))
        population = generateNextPopulation(population, populationSize)
        generation += 1

    for x in population:
        if x.fitness() == 28:            
            if x.parents is not None:
                print("Parents::")
                print(x.parents[0])
                print(x.parents[1])
            print("Solution state: ")
            print(x)


generation 0 Max fitness: 27
generation 1 Max fitness: 27
generation 2 Max fitness: 27
generation 3 Max fitness: 23
generation 4 Max fitness: 25
generation 5 Max fitness: 25
generation 6 Max fitness: 24
generation 7 Max fitness: 27
generation 8 Max fitness: 26
generation 9 Max fitness: 25
generation 10 Max fitness: 25
generation 11 Max fitness: 25
generation 12 Max fitness: 23
generation 13 Max fitness: 23
generation 14 Max fitness: 22
generation 15 Max fitness: 23
generation 16 Max fitness: 22
generation 17 Max fitness: 23
generation 18 Max fitness: 22
generation 19 Max fitness: 23
generation 20 Max fitness: 23
generation 21 Max fitness: 23
generation 22 Max fitness: 23
generation 23 Max fitness: 22
generation 24 Max fitness: 23
generation 25 Max fitness: 23
generation 26 Max fitness: 23
generation 27 Max fitness: 22
generation 28 Max fitness: 24
generation 29 Max fitness: 23
generation 30 Max fitness: 23
generation 31 Max fitness: 24
generation 32 Max fitness: 25
generation 33 Max fi